In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.models import load_model
import pandas as pd
import datetime
import pickle

In [5]:
model=load_model('model.h5')

In [7]:
# loading scaler,label_encoder_gender,one_hot_encoder

with open('label_encode_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder=pickle.load(file)

with open('scalar.pkl','rb') as file:
    scalar=pickle.load(file)

In [9]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000.0
}

In [13]:
input_data['Gender'] = label_encoder_gender.transform([input_data['Gender']])[0]

In [14]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 1,
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000.0,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000.0}

In [30]:
input_encoder=one_hot_encoder.transform([[input_data['Geography']]]).toarray()

g:\annclassification\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [24]:
input_encoder

array([[1., 0., 0.]])

In [28]:
geo_encoder_df=pd.DataFrame(input_encoder,columns=one_hot_encoder.get_feature_names_out(['Geography']))

In [29]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
# Convert input_data dictionary to a DataFrame first
input_df = pd.DataFrame([input_data])

# Drop 'Geography' column and concatenate with encoded geography
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoder_df], axis=1)

input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000.0,2,1,1,50000.0,1.0,0.0,0.0


In [37]:
input_scaled=scalar.transform(input_df)

input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [38]:
prediction=model.predict(input_scaled)

prediction

1/1 [==============================] - 1s 574ms/step


array([[0.02773883]], dtype=float32)

In [39]:
prediction_prob=prediction[0][0]

prediction_prob

0.027738834

In [40]:
if(prediction_prob>0.5):
    print('The customer us likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
